In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import yaml 
import pandas as pd
from sklearn.metrics import accuracy_score
import xgboost as xgb
import pickle
import gc
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.linear_model import RidgeClassifierCV

%matplotlib inline


Using TensorFlow backend.


In [4]:
def dataset_dt_splt(df, st_dt, ed_st):
    return df[(df.date> st_dt) & (df.date <= ed_st)]
def utility(dates: np.ndarray, returns: np.ndarray,
            weights: np.ndarray, actions: np.ndarray) -> np.ndarray:
    gains = pd.DataFrame({
        'date': dates,
        'gain': weights * returns * actions
    })

    p = gains.groupby('date').sum().to_numpy()

    i = len(set(dates))

    t = p.sum() / np.sqrt(np.power(p, 2).sum()) * np.sqrt(250 / i)

    return np.clip(t, a_min=0, a_max=6) * p.sum()

In [5]:
train = pd.read_csv('input/train.csv').astype('float32')
# train = pd.read_csv('/content/drive/MyDrive/janeStreetKaggle/input/train.csv').astype('float32')
## TODO not sure if it's the right thing to do
## very unbalance if include 0 weight, maybe use imblearn.RandomSampler
train = train[train['weight'] != 0]
train['action'] = (train.weight*train.resp>0).astype('int')


In [6]:
## max weight
max_weight = train.weight.max()
max_dt = train.date.max()
feat_cols = [col for col in train.columns if 'feature' in col] + ['weight']
## First Change: Feeding Sample Weight in clf.fit
# train_weight = train['weight']

datasets = {}
splt_dates = [0, 250, 375, 500]

ds_train = dataset_dt_splt(train, splt_dates[0], splt_dates[1])
ds_val = dataset_dt_splt(train, splt_dates[1], splt_dates[2])


print("MinMaxScaler")
sc = MinMaxScaler().fit(ds_train[feat_cols])
scale_clean_X = sc.transform(ds_train[feat_cols])

# print("StandardScaler")
# sc = StandardScaler().fit(ds_train[feat_cols])
# scale_clean_X = sc.transform(ds_train[feat_cols])



print("MeanImputing")

si = SimpleImputer(strategy='mean').fit(scale_clean_X)
scale_clean_X = si.transform(scale_clean_X)

y_train = ds_train.action


gc.collect()


MinMaxScaler
MeanImputing


33

In [7]:
scaled_ds_train = pd.DataFrame(scale_clean_X, columns= feat_cols)
scaled_ds_train['action'] = y_train

In [10]:
import seaborn as sns

In [ ]:
pair_fg = sns.pairplot(ds_train, hue="action", height=1.5)
pair_fg.savefig("pairplot.png", dpi=800)

for i, feat in enumerate(feat_cols):

    print(feat)
    g = sns.displot(ax=ax, x=scale_clean_X[:, i], hue=y_train)
    g.fig.set_size_inches(15,15)
    plt.show()
    